In [1]:
from moccasin import setup_notebook

setup_notebook()

In [2]:
from moccasin.config import get_active_network

active_network = get_active_network()
print(f"Active network: {active_network.name}")

Active network: eth-forked


In [3]:
import boa
from boa.contracts.abi.abi_contract import ABIContract
from moccasin.config import get_active_network, Network

STARTING_ETH_BALANCE = int(1000e18)  # 1000 ETH
STARTING_WETH_BALANCE = int(1e18)  # 1 wETH
STARTING_USDC_BALANCE = int(100e6)  # 100 USDC (6 decimals)


def _add_eth_balance() -> None:
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)


def _add_token_balance(usdc: ABIContract, weth: ABIContract) -> None:
    weth.deposit(value=STARTING_WETH_BALANCE)

    our_address = boa.env.eoa
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
    usdc.configureMinter(our_address, STARTING_USDC_BALANCE)
    usdc.mint(our_address, STARTING_USDC_BALANCE)


def setup_script() -> (ABIContract, ABIContract, ABIContract, ABIContract):
    print("Starting setup script...")

    # Give ourselves some ETH

    # Give ourselves some USDC and wETH
    active_network = get_active_network()

    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")

    if active_network.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc, weth)

    return (usdc, weth)


def moccasin_main():
    (usdc, weth) = setup_script()
    return (usdc, weth)


moccasin_main()

Starting setup script...


(<usdc interface at 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48>,
 <weth interface at 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2>)

In [4]:
usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")

In [5]:
from moccasin.config import get_config

config = get_config()
config.reload()
active_network = get_active_network()

In [6]:
aave_pool_address_provider = active_network.manifest_named("aave_pool_address_provider")
pool_address = aave_pool_address_provider.getPool()
print(f"Aave Pool address: {pool_address}")

Aave Pool address: 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [7]:
config.reload()
active_network = get_active_network()
pool_contract = active_network.manifest_named("pool", address=pool_address)

In [8]:
REFERRAL_CODE = 0

def deposit(pool_contract, token, amount):
    allowed_amount = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amount < amount:
        token.approve(pool_contract.address, amount)
    print(f"Depositing {amount} of token {token.name()} to Aave Pool {pool_contract.address}...")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFERRAL_CODE)

usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)

if usdc_balance > 0:
    deposit(pool_contract, usdc, usdc_balance)

if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)

(
    totalCollateralBase,
    totalDebtBase,
    availableBorrowBase,
    currentLiquidationThreshold,
    ltv,
    healthFactor,
) = pool_contract.getUserAccountData(boa.env.eoa)
print(f"""User account data:
    totalCollateralBase: {totalCollateralBase}
    totalDebtBase: {totalDebtBase}
    availableBorrowBase: {availableBorrowBase}
    currentLiquidationThreshold: {currentLiquidationThreshold}
    ltv: {ltv}
    healthFactor: {healthFactor}
""")

Depositing 100000000 of token USD Coin to Aave Pool 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2...
Depositing 1000000000000000000 of token Wrapped Ether to Aave Pool 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2...
User account data:
    totalCollateralBase: 209741177030
    totalDebtBase: 0
    availableBorrowBase: 168275346331
    currentLiquidationThreshold: 8276
    ltv: 8023
    healthFactor: 115792089237316195423570985008687907853269984665640564039457584007913129639935



## We want 30% of our portfolio in USDC and 70% in WETH.

In [9]:
config.reload()
active_network = get_active_network()
aave_protocol_data_provider = active_network.manifest_named("aave_protocol_data_provider")
a_tokens = aave_protocol_data_provider.getAllATokens()
print(f"All aTokens: {a_tokens}")

All aTokens: [getAllATokens(symbol='aEthWETH', tokenAddress=Address('0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8')), getAllATokens(symbol='aEthwstETH', tokenAddress=Address('0x0B925eD163218f6662a35e0f0371Ac234f9E9371')), getAllATokens(symbol='aEthWBTC', tokenAddress=Address('0x5Ee5bf7ae06D1Be5997A1A72006FE6C607eC6DE8')), getAllATokens(symbol='aEthUSDC', tokenAddress=Address('0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c')), getAllATokens(symbol='aEthDAI', tokenAddress=Address('0x018008bfb33d285247A21d44E50697654f754e63')), getAllATokens(symbol='aEthLINK', tokenAddress=Address('0x5E8C8A7243651DB1384C0dDfDbE39761E8e7E51a')), getAllATokens(symbol='aEthAAVE', tokenAddress=Address('0xA700b4eB416Be35b2911fd5Dee80678ff64fF6C9')), getAllATokens(symbol='aEthcbETH', tokenAddress=Address('0x977b6fc5dE62598B08C85AC8Cf2b745874E8b78c')), getAllATokens(symbol='aEthUSDT', tokenAddress=Address('0x23878914EFE38d27C4D67Ab83ed1b93A74D4086a')), getAllATokens(symbol='aEthrETH', tokenAddress=Address('0xCc9EE9483

In [10]:
for a_token in a_tokens:
    if "WETH" in a_token[0]:
        a_weth = active_network.manifest_named("usdc", address = a_token[1])
    if "USDC" in a_token[0]:
        a_usdc = active_network.manifest_named("usdc", address = a_token[1])

print(a_usdc)
print(a_weth)

<usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>
<usdc interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>


In [11]:
# Get how much they are worth, to figure out what our portfolio portions are in USD.
a_usdc_balance = a_usdc.balanceOf(boa.env.eoa) # 6 decimals
a_weth_balance = a_weth.balanceOf(boa.env.eoa) # 18 decimals

a_usdc_balance_normalized = a_usdc_balance / 1e6
a_weth_balance_normalized = a_weth_balance / 1e18
print(f"USDC balance: {a_usdc_balance_normalized}")
print(f"WETH balance: {a_weth_balance_normalized}")

USDC balance: 99.999999
WETH balance: 1.0


In [12]:
# Chainlink ETH/USD price feed address: 0x5f4eC3Df9cbd43714FE2740f5E3616155c5b8419
# USDC/USD price feed address: 0x8fFfFfd4AfB6115b954Bd326cbe7B4BA576818f6

def get_price(feed_name) -> float:
    active_network = get_active_network()
    price_feed = active_network.manifest_named(feed_name)
    # latestAnswer is deprecated, but it's simpler to use for this example than latestRoundData.
    # In production code, you should use latestRoundData and handle the possibility of stale data.
    price = price_feed.latestAnswer()
    decimals = price_feed.decimals()
    normalized_price = price / (10 ** decimals)
    return normalized_price

config.reload()
eth_usd_price = get_price("eth_usd")
usdc_usd_price = get_price("usdc_usd")
print(f"ETH/USD price: {eth_usd_price}")
print(f"USDC/USD price: {usdc_usd_price}")

ETH/USD price: 1996.02699833
USDC/USD price: 0.99992


In [13]:
usdc_value = a_usdc_balance_normalized * usdc_usd_price
weth_value = a_weth_balance_normalized * eth_usd_price
total_value = usdc_value + weth_value
usdc_portion = usdc_value / total_value
weth_portion = weth_value / total_value

target_usdc_portion = 0.3
target_weth_portion = 0.7

print(f"USDC value: {usdc_value}")
print(f"WETH value: {weth_value}")
print(f"Total value: {total_value}")
print(f"USDC portion: {usdc_portion}")
print(f"WETH portion: {weth_portion}")

USDC value: 99.99199900008
WETH value: 1996.02699833
Total value: 2096.01899733008
USDC portion: 0.04770567400746383
WETH portion: 0.9522943259925363


In [14]:
BUFFER = 0.1
needs_rebalancing = (
    abs(usdc_portion - target_usdc_portion) > BUFFER
    or abs(weth_portion - target_weth_portion) > BUFFER
)
print (f"Needs rebalancing: {needs_rebalancing}")

Needs rebalancing: True


In [15]:
a_weth.approve(pool_contract.address, a_weth.balanceOf(boa.env.eoa))
pool_contract.withdraw(weth.address, a_weth.balanceOf(boa.env.eoa), boa.env.eoa)

def print_token_balances():
    print(f"USDC balance: {usdc.balanceOf(boa.env.eoa)}")
    print(f"WETH balance: {weth.balanceOf(boa.env.eoa)}")
    print(f"aUSDC balance: {a_usdc.balanceOf(boa.env.eoa)}")
    print(f"aWETH balance: {a_weth.balanceOf(boa.env.eoa)}")

print_token_balances()

USDC balance: 0
WETH balance: 999999999999999999
aUSDC balance: 99999999
aWETH balance: 0


In [16]:
usdc_data = {"balance": a_usdc_balance_normalized, "price": usdc_usd_price, "contract": usdc}
weth_data = {"balance": a_weth_balance_normalized, "price": eth_usd_price, "contract": weth}
target_allocations = {"usdc": target_usdc_portion, "weth": target_weth_portion}

def calculate_rebalancing_trades(
    usdc_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    weth_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    target_allocations: dict[str, float],  # {"usdc": 0.3, "weth": 0.7}
) -> dict[str, dict]:
    """
    Calculate the trades needed to rebalance a portfolio of USDC and WETH.

    Args:
        usdc_data: Dict containing USDC balance, price and contract
        weth_data: Dict containing WETH balance, price and contract
        target_allocations: Dict of token symbol to target allocation (must sum to 1)

    Returns:
        Dict of token symbol to dict containing contract and trade amount:
            {"usdc": {"contract": Contract, "trade": int},
             "weth": {"contract": Contract, "trade": int}}
    """
    # Calculate current values
    usdc_value = usdc_data["balance"] * usdc_data["price"]
    weth_value = weth_data["balance"] * weth_data["price"]
    total_value = usdc_value + weth_value

    # Calculate target values
    target_usdc_value = total_value * target_allocations["usdc"]
    target_weth_value = total_value * target_allocations["weth"]

    # Calculate trades needed in USD
    usdc_trade_usd = target_usdc_value - usdc_value
    weth_trade_usd = target_weth_value - weth_value

    # Convert to token amounts
    return {
        "usdc": {
            "contract": usdc_data["contract"],
            "trade": usdc_trade_usd / usdc_data["price"],
        },
        "weth": {
            "contract": weth_data["contract"],
            "trade": weth_trade_usd / weth_data["price"],
        },
    }

trades = calculate_rebalancing_trades(usdc_data, weth_data, target_allocations)
print(f"Trades needed to rebalance: {trades}")

weth_to_sell = trades["weth"]["trade"]



Trades needed to rebalance: {'usdc': {'contract': <usdc interface at 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48>, 'trade': 528.8560086796383}, 'weth': {'contract': <weth interface at 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2>, 'trade': -0.2649331400033078}}


In [17]:
    # struct ExactInputSingleParams {
    #     address tokenIn;
    #     address tokenOut;
    #     uint24 fee;
    #     address recipient;
    #     uint256 deadline;  # actually there's no deadline arg, odd
    #     uint256 amountIn;
    #     uint256 amountOutMinimum;
    #     uint160 sqrtPriceLimitX96;
    # }

config.reload()
active_network = get_active_network()
uniswap_swap_router = active_network.manifest_named("uniswap_swap_router")

amount_weth = abs(int(weth_to_sell * (10 ** 18)))  # Convert to 18 decimals
weth.approve(uniswap_swap_router.address, amount_weth)

min_out = int(trades["usdc"]["trade"] * (10 ** 6) * 0.90)  # Convert to 6 decimals and set slippage tolerance to 5%

uniswap_swap_router.exactInputSingle(
    (
        weth.address,
        usdc.address,
        3000,  # fee tier of the pool to use (0.3%)
        boa.env.eoa,
        amount_weth,
        min_out,
        0
    )
)

print_token_balances()

USDC balance: 526510586
WETH balance: 735066859996692191
aUSDC balance: 99999999
aWETH balance: 0


In [ ]:
amount = usdc.balanceOf(boa.env.eoa)
deposit(pool_contract, usdc, amount)



Depositing 526510586 of token USD Coin to Aave Pool 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2...
USDC balance: 0
WETH balance: 735066859996692191
aUSDC balance: 626510585
aWETH balance: 0
USDC balance: 626.510585
WETH balance: 0.0
USDC portion: 1.0
WETH portion: 0.0


In [19]:
amount = weth.balanceOf(boa.env.eoa)
deposit(pool_contract, weth, amount)

Depositing 735066859996692191 of token Wrapped Ether to Aave Pool 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2...


In [20]:
print_token_balances()

a_usdc_balance = a_usdc.balanceOf(boa.env.eoa) # 6 decimals
a_weth_balance = a_weth.balanceOf(boa.env.eoa) # 18 decimals

a_usdc_balance_normalized = a_usdc_balance / 1e6
a_weth_balance_normalized = a_weth_balance / 1e18
print(f"USDC balance: {a_usdc_balance_normalized}")
print(f"WETH balance: {a_weth_balance_normalized}")

usdc_value = a_usdc_balance_normalized * usdc_usd_price
weth_value = a_weth_balance_normalized * eth_usd_price
total_value = usdc_value + weth_value
usdc_portion = usdc_value / total_value
weth_portion = weth_value / total_value

print(f"USDC portion: {usdc_portion}")
print(f"WETH portion: {weth_portion}")

USDC balance: 0
WETH balance: 0
aUSDC balance: 626510585
aWETH balance: 735066859996692190
USDC balance: 626.510585
WETH balance: 0.7350668599966922
USDC portion: 0.29921589286657274
WETH portion: 0.7007841071334274
